In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
!pip install folium==0.7.0

In [4]:
spacex_df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

In [5]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [6]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [15]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
launch_sites_dict_longlat = launch_sites_df.set_index('Launch Site').T.to_dict('list')
launch_sites_dict_longlat

{'CCAFS LC-40': [28.56230197, -80.57735648],
 'CCAFS SLC-40': [28.56319718, -80.57682003],
 'KSC LC-39A': [28.57325457, -80.64689529],
 'VAFB SLC-4E': [34.63283416, -120.6107455]}

In [16]:
for x, y in launch_sites_dict_longlat.items():
    # Create a blue circle at site coordinates with a popup label showing its name
    circle = folium.Circle(y, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(x + ' Launch Site'))
    # Create a blue circle site coordinates with a icon showing its name
    marker = folium.map.Marker(
    y,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:(#ADD8E6);"><b>%s</b></div>' % x,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)  

In [17]:
site_map

# Task 2: Mark the success/failed launches for each site on the map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
marker_cluster = MarkerCluster()

In [18]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [21]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker([record[1], record[2]], icon=folium.Icon(color='white', icon_color=record[4]))
    marker_cluster.add_child(marker)

site_map

In [20]:
import folium
from folium.features import DivIcon

# Define coordinates for the launch sites
nasa_coordinate = [29.5593, -95.0894]
other_site_coordinate = [28.5721, -80.6480]  # Example coordinates for another site

# Define the data for launches (hypothetical)
launches = {
    'NASA Johnson Space Center': {
        'coordinate': nasa_coordinate,
        'launches': [
            {'status': 'success', 'name': 'Launch 1'},
            {'status': 'failed', 'name': 'Launch 2'},
        ]
    },
    'Other Launch Site': {
        'coordinate': other_site_coordinate,
        'launches': [
            {'status': 'success', 'name': 'Launch 3'},
            {'status': 'failed', 'name': 'Launch 4'},
        ]
    }
}

# Create a map centered around the average coordinates
site_map = folium.Map(location=[29.5, -85.0], zoom_start=5)

# Function to get color based on launch status
def get_color(status):
    return 'green' if status == 'success' else 'red'

# Add markers for each site and its launches
for site, data in launches.items():
    coordinate = data['coordinate']
    
    # Add a circle for the site
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_child(folium.Popup(site)).add_to(site_map)
    
    # Add a label marker for the site
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; color:blue;"><b>{site}</b></div>'
        )
    ).add_to(site_map)
    
    # Add markers for each launch
    for launch in data['launches']:
        launch_status = launch['status']
        launch_name = launch['name']
        
        folium.Marker(
            location=coordinate,
            popup=folium.Popup(f'{launch_name} - {launch_status}'),
            icon=folium.Icon(color=get_color(launch_status))
        ).add_to(site_map)

# Save the map to an HTML file
site_map


# TASK 3: Calculate the distances between a launch site to its proximities


In [13]:
import folium
import math
from folium.features import DivIcon
from geopy.distance import geodesic

# Define coordinates for the launch sites
nasa_coordinate = [29.5593, -95.0894]
other_site_coordinate = [28.5721, -80.6480]  # Example coordinates for another site

# Define the data for launches (hypothetical)
launches = {
    'NASA Johnson Space Center': {
        'coordinate': nasa_coordinate,
        'launches': [
            {'status': 'success', 'name': 'Launch 1'},
            {'status': 'failed', 'name': 'Launch 2'},
        ]
    },
    'Other Launch Site': {
        'coordinate': other_site_coordinate,
        'launches': [
            {'status': 'success', 'name': 'Launch 3'},
            {'status': 'failed', 'name': 'Launch 4'},
        ]
    }
}

# Define some proximities (hypothetical data)
proximities = {
    'Proximity A': [29.7604, -95.3698],  # Example coordinates for Houston
    'Proximity B': [29.4241, -98.4936],  # Example coordinates for San Antonio
    'Proximity C': [30.2672, -97.7431]   # Example coordinates for Austin
}

# Create a map centered around the average coordinates
site_map = folium.Map(location=[29.5, -85.0], zoom_start=5)

# Function to get color based on launch status
def get_color(status):
    return 'green' if status == 'success' else 'red'

# Add markers for each site and its launches
for site, data in launches.items():
    coordinate = data['coordinate']
    
    # Add a circle for the site
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_child(folium.Popup(site)).add_to(site_map)
    
    # Add a label marker for the site
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; color:blue;"><b>{site}</b></div>'
        )
    ).add_to(site_map)
    
    # Add markers for each launch
    for launch in data['launches']:
        launch_status = launch['status']
        launch_name = launch['name']
        
        folium.Marker(
            location=coordinate,
            popup=folium.Popup(f'{launch_name} - {launch_status}'),
            icon=folium.Icon(color=get_color(launch_status))
        ).add_to(site_map)
    
    # Calculate and add markers for proximities
    for proximity_name, proximity_coord in proximities.items():
        distance = geodesic(coordinate, proximity_coord).kilometers
        folium.Marker(
            location=proximity_coord,
            popup=folium.Popup(f'{proximity_name}: {distance:.2f} km from {site}'),
            icon=folium.Icon(color='purple')
        ).add_to(site_map)

# Save the map to an HTML file
site_map


In [24]:
!pip install geopy

In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [26]:
distance_coastline = calculate_distance(34.632834161782775, -120.61074553068327, 34.63709, -120.62450)

In [27]:
distance_marker = folium.Marker([ 34.63709, -120.62450], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline)))

In [28]:
lines=folium.PolyLine(locations=[[34.632834161782775, -120.61074553068327], [ 34.63709, -120.62450]], weight=1)
site_map.add_child(distance_marker)
site_map.add_child(lines)

In [29]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Create a marker with distance to a closest city, railway, highway, etc.
distance_railways = calculate_distance(34.632834161782775, -120.61074553068327, 34.63487, -120.62448)
distance_marker_railways = folium.Marker([34.63487, -120.62448], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railways)))
site_map.add_child(distance_marker_railways)

# Draw a line between the marker to the launch site
lines_railways=folium.PolyLine(locations=[[34.632834161782775, -120.61074553068327], [34.63487, -120.62448]], weight=1.2)
site_map.add_child(lines_railways)